<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/experimental/ML_model(Resampling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
#!pip install yfinance

In [161]:
import yfinance as yf
import numpy as np
import pandas as pd


In [162]:
nifty = yf.download('^nsei',start='2014-01-01',end='2020-12-31')

[*********************100%***********************]  1 of 1 completed


In [163]:
nifty.tail(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-29,13910.349609,13967.599609,13859.900391,13932.599609,13932.599609,439600
2020-12-30,13980.900391,13997.000000,13864.950195,13981.950195,13981.950195,380700


In [164]:
nifty['close_2%']=np.abs(nifty['Adj Close'].pct_change()*100).fillna(0)

In [165]:
test=pd.DataFrame(data=nifty['close_2%'].values,columns=['sum'])

In [166]:
nifty['cumsum']=0
nifty=nifty.reset_index()

In [167]:
for index , val in enumerate(nifty['close_2%']):
  
  if index==0:
    nifty.loc[index,'cumsum']=val
  else:
    if nifty['cumsum'].iloc[index-1]>2:
      nifty.loc[index,'cumsum']=val
    else:
      nifty.loc[index, 'cumsum']=val+nifty['cumsum'].iloc[index-1]    

In [168]:
#test_f=pd.DataFrame({'Date':nifty.index, 'sum':test['cumsum'].values})
#pd.DataFrame({'Date':df.index,'spread':(df['Adj Close'].values - mean)})

In [171]:
nifty['diff']=nifty['close_2%']-nifty['cumsum']

In [174]:
nifty.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,close_2%,cumsum,diff
0,2014-01-02,6301.250000,6358.299805,6211.299805,6221.149902,6221.149902,158100,0.000000,0.000000,0.000000
1,2014-01-03,6194.549805,6221.700195,6171.250000,6211.149902,6211.149902,139000,0.160742,0.160742,0.000000
2,2014-01-06,6220.850098,6224.700195,6170.250000,6191.450195,6191.450195,118300,0.317167,0.477909,-0.160742
3,2014-01-07,6203.899902,6221.500000,6144.750000,6162.250000,6162.250000,138600,0.471621,0.949530,-0.477909
4,2014-01-08,6178.049805,6192.100098,6160.350098,6174.600098,6174.600098,146900,0.200415,1.149945,-0.949530
5,2014-01-09,6181.700195,6188.049805,6148.250000,6168.350098,6168.350098,150100,0.101221,1.251167,-1.149945
6,2014-01-10,6178.850098,6239.100098,6139.600098,6171.450195,6171.450195,159900,0.050258,1.301425,-1.251167
7,2014-01-13,6189.549805,6288.200195,6189.549805,6272.750000,6272.750000,135000,1.641426,2.942851,-1.301425
8,2014-01-14,6260.250000,6280.350098,6234.149902,6241.850098,6241.850098,110200,0.492605,0.492605,0.000000
9,2014-01-15,6265.950195,6325.200195,6265.299805,6320.899902,6320.899902,145900,1.266448,1.759054,-0.492605


In [182]:
final_data=nifty.iloc[nifty[nifty['diff']==0].index-1][1:]

In [184]:
days_out=1

In [185]:
final_data['Signal_Flag'] = final_data['Close'].transform(lambda x : np.sign(x.diff(days_out)))

In [187]:
final_data=final_data.dropna()

In [188]:
final_data

,Date,Open,High,Low,Close,Adj Close,Volume,close_2%,cumsum,diff,Signal_Flag
7,2014-01-13,6189.549805,6288.200195,6189.549805,6272.750000,6272.750000,135000,1.641426,2.942851,-1.301425,1.0
11,2014-01-17,6306.250000,6327.100098,6246.350098,6261.649902,6261.649902,167700,0.906012,2.696707,-1.790695,-1.0
16,2014-01-24,6301.649902,6331.450195,6263.899902,6266.750000,6266.750000,160300,1.243370,2.579192,-1.335822,1.0
17,2014-01-27,6186.299805,6188.549805,6130.250000,6135.850098,6135.850098,190400,2.088800,2.088800,0.000000,-1.0
22,2014-02-03,6058.799805,6074.850098,5994.450195,6001.799805,6001.799805,134900,1.440187,2.715307,-1.275120,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
1695,2020-12-08,13393.849609,13435.450195,13311.049805,13392.950195,13392.950195,543200,0.278533,2.150226,-1.871693,1.0
1700,2020-12-15,13547.200195,13589.650391,13447.049805,13567.849609,13567.849609,497600,0.071538,2.055172,-1.983634,1.0
1704,2020-12-21,13741.900391,13777.500000,13131.450195,13328.400391,13328.400391,687300,3.140495,4.555337,-1.414842,-1.0
1706,2020-12-23,13473.500000,13619.450195,13432.200195,13601.099609,13601.099609,458700,1.001016,2.035644,-1.034628,1.0


In [189]:
final_data.Signal_Flag.value_counts()

 1.0    252
-1.0    207
 0.0      1
Name: Signal_Flag, dtype: int64

# Features Creation and Modelling Part remaining